# At rest dataset SVM

In [1]:
import numpy as np
import scipy
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import sklearn.metrics
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from sklearn.preprocessing import StandardScaler, LabelEncoder
from PIL import Image
import time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd

In [2]:
def create_array(df):
    scaled = True
    arr = np.array(df["Signal"])
    arr_numpy = []
    for i in range(len(arr)):
        arr[i] = arr[i].replace("[","")
        arr[i] = arr[i].replace("\n","")
        arr[i] = arr[i].replace("]","")
        float_list = []
        for item in arr[i].split():  
            float_list.append(float(item))
        arr_numpy.append(float_list)
        
    arr_numpy = np.array(arr_numpy)
    if scaled:
        scaler = StandardScaler()
        arr_numpy = scaler.fit_transform(arr_numpy.T).T
    return arr_numpy , np.array(df["Label"])

# ECG

### SVM ECG FILTERED

In [3]:
ecg = pd.read_csv("../DATA_CSV/ECG_filtered_noscaled.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ecg_df, test_ecg_df = train_test_split(ecg, test_size=0.2, stratify=ecg['Label'], random_state=123)

ecg_mat_train, label_ecg_train = create_array(train_ecg_df) 
ecg_mat_test, label_ecg_test  = create_array(test_ecg_df) 

X_train = ecg_mat_train
y_train = label_ecg_train
X_test = ecg_mat_test
y_test = label_ecg_test

In [33]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

RESULTS FOR LINEAR KERNEL
Best parameters set found:
{'C': 0.1, 'kernel': 'linear'}
Score with best parameters:
0.9948835568101624
All scores on the grid:


,params,mean_test_score
0,"{'C': 0.001, 'kernel': 'linear'}",0.964273
1,"{'C': 0.01, 'kernel': 'linear'}",0.989855
2,"{'C': 0.1, 'kernel': 'linear'}",0.994884
3,"{'C': 1, 'kernel': 'linear'}",0.994001
4,"{'C': 10, 'kernel': 'linear'}",0.991796
5,"{'C': 100, 'kernel': 'linear'}",0.989150


In [34]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

RESULTS FOR POLY DEGREE=2 KERNEL
Best parameters set found:
{'C': 0.5, 'gamma': 0.05}
Score with best parameters:
0.9982357092448836

All scores on the grid:


,params,mean_test_score
0,"{'C': 0.05, 'gamma': 0.05}",0.998059
1,"{'C': 0.05, 'gamma': 0.5}",0.998147
2,"{'C': 0.05, 'gamma': 5.0}",0.998147
3,"{'C': 0.5, 'gamma': 0.05}",0.998236
4,"{'C': 0.5, 'gamma': 0.5}",0.998147
5,"{'C': 0.5, 'gamma': 5.0}",0.998147
6,"{'C': 5, 'gamma': 0.05}",0.998147
7,"{'C': 5, 'gamma': 0.5}",0.998147
8,"{'C': 5, 'gamma': 5.0}",0.998147


In [35]:
# parameters for poly with higher degree kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of higher degree kernel
degree = 4

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=degree), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=', degree, ' KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

RESULTS FOR POLY DEGREE= 4  KERNEL
Best parameters set found:
{'C': 0.05, 'gamma': 0.05}
Score with best parameters:
0.9977946365561046

All scores on the grid:


,params,mean_test_score
0,"{'C': 0.05, 'gamma': 0.05}",0.997795
1,"{'C': 0.05, 'gamma': 0.5}",0.997795
2,"{'C': 0.05, 'gamma': 5.0}",0.997795
3,"{'C': 0.5, 'gamma': 0.05}",0.997795
4,"{'C': 0.5, 'gamma': 0.5}",0.997795
5,"{'C': 0.5, 'gamma': 5.0}",0.997795
6,"{'C': 5, 'gamma': 0.05}",0.997795
7,"{'C': 5, 'gamma': 0.5}",0.997795
8,"{'C': 5, 'gamma': 5.0}",0.997795


In [36]:
# parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='rbf'), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

RESULTS FOR rbf KERNEL
Best parameters set found:
{'C': 5, 'gamma': 0.005}
Score with best parameters:
0.9979710656316161

All scores on the grid:


,params,mean_test_score
0,"{'C': 0.5, 'gamma': 0.005}",0.996119
1,"{'C': 0.5, 'gamma': 0.05}",0.997706
2,"{'C': 0.5, 'gamma': 0.5}",0.988003
3,"{'C': 0.5, 'gamma': 5}",0.302929
4,"{'C': 5, 'gamma': 0.005}",0.997971
5,"{'C': 5, 'gamma': 0.05}",0.997971
6,"{'C': 5, 'gamma': 0.5}",0.991884
7,"{'C': 5, 'gamma': 5}",0.498677
8,"{'C': 50, 'gamma': 0.005}",0.997883
9,"{'C': 50, 'gamma': 0.05}",0.997971


In [4]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='poly',degree=2, C=0.5, gamma=0.05).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))

Execution time =  0.8082236576080323
Highest score =  0.9992945326278659


### SVM ECG FILTERED PCA

In [5]:
ecg = pd.read_csv("../DATA_CSV/ECG_filtered_noscaled.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ecg_df, test_ecg_df = train_test_split(ecg, test_size=0.2, stratify=ecg['Label'], random_state=123)

ecg_mat_train, label_ecg_train = create_array(train_ecg_df) 
ecg_mat_test, label_ecg_test  = create_array(test_ecg_df) 

num_components = 0.95
pca2 = PCA(n_components = num_components)
pca2.fit(ecg_mat_train)
ecg_mat_train = pca2.transform(ecg_mat_train)
ecg_mat_test = pca2.transform(ecg_mat_test)

X_train = ecg_mat_train
y_train = label_ecg_train
X_test = ecg_mat_test
y_test = label_ecg_test

In [11]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

RESULTS FOR LINEAR KERNEL
Best parameters set found:
{'C': 100, 'kernel': 'linear'}
Score with best parameters:
0.912755822159492
All scores on the grid:


,params,mean_test_score
0,"{'C': 0.001, 'kernel': 'linear'}",0.870501
1,"{'C': 0.01, 'kernel': 'linear'}",0.908963
2,"{'C': 0.1, 'kernel': 'linear'}",0.912315
3,"{'C': 1, 'kernel': 'linear'}",0.912579
4,"{'C': 10, 'kernel': 'linear'}",0.912579
5,"{'C': 100, 'kernel': 'linear'}",0.912756


In [12]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

RESULTS FOR POLY DEGREE=2 KERNEL
Best parameters set found:
{'C': 0.05, 'gamma': 5.0}
Score with best parameters:
0.9643613267466478

All scores on the grid:


,params,mean_test_score
0,"{'C': 0.05, 'gamma': 0.05}",0.919725
1,"{'C': 0.05, 'gamma': 0.5}",0.960039
2,"{'C': 0.05, 'gamma': 5.0}",0.964361
3,"{'C': 0.5, 'gamma': 0.05}",0.944072
4,"{'C': 0.5, 'gamma': 0.5}",0.964008
5,"{'C': 0.5, 'gamma': 5.0}",0.964273
6,"{'C': 5, 'gamma': 0.05}",0.959951
7,"{'C': 5, 'gamma': 0.5}",0.964361
8,"{'C': 5, 'gamma': 5.0}",0.964097


In [ ]:
# # parameters for poly with higher degree kernel
# parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

# #run SVM with poly of higher degree kernel
# degree = 4

# svc = SVC()
# clf = GridSearchCV(SVC(kernel='poly', degree=degree), parameters, cv=4, return_train_score=True)
# clf.fit(X_train, y_train)

# print ('RESULTS FOR POLY DEGREE=', degree, ' KERNEL')

# print("Best parameters set found:")
# print(clf.best_params_)

# print("Score with best parameters:")
# print(clf.best_score_)

# print("\nAll scores on the grid:")
# df=pd.DataFrame(clf.cv_results_)
# df[['params', 'mean_test_score']]

In [5]:
# parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='rbf'), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

RESULTS FOR rbf KERNEL
Best parameters set found:
{'C': 500, 'gamma': 0.05}
Score with best parameters:
0.9879146083274524

All scores on the grid:


,params,mean_test_score
0,"{'C': 0.5, 'gamma': 0.005}",0.946189
1,"{'C': 0.5, 'gamma': 0.05}",0.969831
2,"{'C': 0.5, 'gamma': 0.5}",0.979799
3,"{'C': 0.5, 'gamma': 5}",0.799665
4,"{'C': 5, 'gamma': 0.005}",0.963744
5,"{'C': 5, 'gamma': 0.05}",0.983063
6,"{'C': 5, 'gamma': 0.5}",0.985356
7,"{'C': 5, 'gamma': 5}",0.903934
8,"{'C': 50, 'gamma': 0.005}",0.977064
9,"{'C': 50, 'gamma': 0.05}",0.986591


In [6]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='rbf', C=500, gamma=0.05).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))

Execution time =  0.553218309879303
Highest score =  0.9887125220458554


### SVM ECG nonFILTERED

In [7]:
ecg = pd.read_csv("../DATA_CSV/ECG_nofiltered_noscaled.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ecg_df, test_ecg_df = train_test_split(ecg, test_size=0.2, stratify=ecg['Label'], random_state=123)

ecg_mat_train, label_ecg_train = create_array(train_ecg_df) 
ecg_mat_test, label_ecg_test  = create_array(test_ecg_df) 

X_train = ecg_mat_train
y_train = label_ecg_train
X_test = ecg_mat_test
y_test = label_ecg_test

In [8]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

RESULTS FOR LINEAR KERNEL
Best parameters set found:
{'C': 1, 'kernel': 'linear'}
Score with best parameters:
0.9933868030835591
All scores on the grid:


,params,mean_test_score
0,"{'C': 0.001, 'kernel': 'linear'}",0.964730
1,"{'C': 0.01, 'kernel': 'linear'}",0.988184
2,"{'C': 0.1, 'kernel': 'linear'}",0.993122
3,"{'C': 1, 'kernel': 'linear'}",0.993387
4,"{'C': 10, 'kernel': 'linear'}",0.991271
5,"{'C': 100, 'kernel': 'linear'}",0.988096


In [9]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

RESULTS FOR POLY DEGREE=2 KERNEL
Best parameters set found:
{'C': 0.05, 'gamma': 0.5}
Score with best parameters:
0.9983246393683629

All scores on the grid:


,params,mean_test_score
0,"{'C': 0.05, 'gamma': 0.05}",0.998060
1,"{'C': 0.05, 'gamma': 0.5}",0.998325
2,"{'C': 0.05, 'gamma': 5.0}",0.998325
3,"{'C': 0.5, 'gamma': 0.05}",0.998236
4,"{'C': 0.5, 'gamma': 0.5}",0.998325
5,"{'C': 0.5, 'gamma': 5.0}",0.998325
6,"{'C': 5, 'gamma': 0.05}",0.998325
7,"{'C': 5, 'gamma': 0.5}",0.998325
8,"{'C': 5, 'gamma': 5.0}",0.998325


In [10]:
# parameters for poly with higher degree kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of higher degree kernel
degree = 4

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=degree), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=', degree, ' KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

RESULTS FOR POLY DEGREE= 4  KERNEL
Best parameters set found:
{'C': 0.05, 'gamma': 0.05}
Score with best parameters:
0.9980601512924033

All scores on the grid:


,params,mean_test_score
0,"{'C': 0.05, 'gamma': 0.05}",0.99806
1,"{'C': 0.05, 'gamma': 0.5}",0.99806
2,"{'C': 0.05, 'gamma': 5.0}",0.99806
3,"{'C': 0.5, 'gamma': 0.05}",0.99806
4,"{'C': 0.5, 'gamma': 0.5}",0.99806
5,"{'C': 0.5, 'gamma': 5.0}",0.99806
6,"{'C': 5, 'gamma': 0.05}",0.99806
7,"{'C': 5, 'gamma': 0.5}",0.99806
8,"{'C': 5, 'gamma': 5.0}",0.99806


In [5]:
# parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='rbf', max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3

RESULTS FOR rbf KERNEL
Best parameters set found:
{'C': 50, 'gamma': 0.005}
Score with best parameters:
0.9980601512924033

All scores on the grid:


D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


,params,mean_test_score
0,"{'C': 0.5, 'gamma': 0.005}",0.995679
1,"{'C': 0.5, 'gamma': 0.05}",0.997267
2,"{'C': 0.5, 'gamma': 0.5}",0.989507
3,"{'C': 0.5, 'gamma': 5}",0.803280
4,"{'C': 5, 'gamma': 0.005}",0.998060
5,"{'C': 5, 'gamma': 0.05}",0.997707
6,"{'C': 5, 'gamma': 0.5}",0.992329
7,"{'C': 5, 'gamma': 5}",0.882638
8,"{'C': 50, 'gamma': 0.005}",0.998060
9,"{'C': 50, 'gamma': 0.05}",0.997707


In [8]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='poly',degree=2, C=0.05, gamma=0.5).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))

Execution time =  0.6642652869224548
Highest score =  0.9982369534555712


### SVM ECG noFILTERED PCA

In [9]:
ecg = pd.read_csv("../DATA_CSV/ECG_nofiltered_noscaled.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ecg_df, test_ecg_df = train_test_split(ecg, test_size=0.2, stratify=ecg['Label'], random_state=123)

ecg_mat_train, label_ecg_train = create_array(train_ecg_df) 
ecg_mat_test, label_ecg_test  = create_array(test_ecg_df) 

num_components = 0.95
pca2 = PCA(n_components = num_components)
pca2.fit(ecg_mat_train)
ecg_mat_train = pca2.transform(ecg_mat_train)
ecg_mat_test = pca2.transform(ecg_mat_test)

X_train = ecg_mat_train
y_train = label_ecg_train
X_test = ecg_mat_test
y_test = label_ecg_test

In [8]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

RESULTS FOR LINEAR KERNEL
Best parameters set found:
{'C': 10, 'kernel': 'linear'}
Score with best parameters:
0.9162332743785494
All scores on the grid:


,params,mean_test_score
0,"{'C': 0.001, 'kernel': 'linear'}",0.891103
1,"{'C': 0.01, 'kernel': 'linear'}",0.914382
2,"{'C': 0.1, 'kernel': 'linear'}",0.915440
3,"{'C': 1, 'kernel': 'linear'}",0.915881
4,"{'C': 10, 'kernel': 'linear'}",0.916233
5,"{'C': 100, 'kernel': 'linear'}",0.915881


In [9]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2, max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3

RESULTS FOR POLY DEGREE=2 KERNEL
Best parameters set found:
{'C': 0.05, 'gamma': 0.5}
Score with best parameters:
0.9660529274657155

All scores on the grid:


D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


,params,mean_test_score
0,"{'C': 0.05, 'gamma': 0.05}",0.937308
1,"{'C': 0.05, 'gamma': 0.5}",0.966053
2,"{'C': 0.05, 'gamma': 5.0}",0.755840
3,"{'C': 0.5, 'gamma': 0.05}",0.956971
4,"{'C': 0.5, 'gamma': 0.5}",0.891543
5,"{'C': 0.5, 'gamma': 5.0}",0.763956
6,"{'C': 5, 'gamma': 0.05}",0.963760
7,"{'C': 5, 'gamma': 0.5}",0.799050
8,"{'C': 5, 'gamma': 5.0}",0.763956


In [10]:
# parameters for poly with higher degree kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of higher degree kernel
degree = 4

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=degree, max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=', degree, ' KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3

RESULTS FOR POLY DEGREE= 4  KERNEL
Best parameters set found:
{'C': 0.5, 'gamma': 0.05}
Score with best parameters:
0.9688739885025733

All scores on the grid:


D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


,params,mean_test_score
0,"{'C': 0.05, 'gamma': 0.05}",0.958646
1,"{'C': 0.05, 'gamma': 0.5}",0.891719
2,"{'C': 0.05, 'gamma': 5.0}",0.875758
3,"{'C': 0.5, 'gamma': 0.05}",0.968874
4,"{'C': 0.5, 'gamma': 0.5}",0.832022
5,"{'C': 0.5, 'gamma': 5.0}",0.875758
6,"{'C': 5, 'gamma': 0.05}",0.920637
7,"{'C': 5, 'gamma': 0.5}",0.832022
8,"{'C': 5, 'gamma': 5.0}",0.875758


In [11]:
# parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='rbf', max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3

RESULTS FOR rbf KERNEL
Best parameters set found:
{'C': 5, 'gamma': 0.5}
Score with best parameters:
0.988008018596876

All scores on the grid:


D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


,params,mean_test_score
0,"{'C': 0.5, 'gamma': 0.005}",0.954237
1,"{'C': 0.5, 'gamma': 0.05}",0.974429
2,"{'C': 0.5, 'gamma': 0.5}",0.981042
3,"{'C': 0.5, 'gamma': 5}",0.893837
4,"{'C': 5, 'gamma': 0.005}",0.970726
5,"{'C': 5, 'gamma': 0.05}",0.985892
6,"{'C': 5, 'gamma': 0.5}",0.988008
7,"{'C': 5, 'gamma': 5}",0.936690
8,"{'C': 50, 'gamma': 0.005}",0.981395
9,"{'C': 50, 'gamma': 0.05}",0.987832


In [10]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='rbf', C=5, gamma=0.5).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))

Execution time =  1.4997012972831727
Highest score =  0.9908321579689704


# PPG

### SVM PPG FILTERED

In [11]:
ppg = pd.read_csv("../DATA_CSV/PPG_filtered_noscaled.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ppg_df, test_ppg_df = train_test_split(ppg, test_size=0.2, stratify=ppg['Label'], random_state=123)

ppg_mat_train, label_ppg_train = create_array(train_ppg_df) 
ppg_mat_test, label_ppg_test   = create_array(test_ppg_df) 

X_train = ppg_mat_train
y_train = label_ppg_train
X_test = ppg_mat_test
y_test = label_ppg_test

from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

In [13]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

RESULTS FOR LINEAR KERNEL
Best parameters set found:
{'C': 10, 'kernel': 'linear'}
Score with best parameters:
0.8099347291272042
All scores on the grid:


,params,mean_test_score
0,"{'C': 0.001, 'kernel': 'linear'}",0.495290
1,"{'C': 0.01, 'kernel': 'linear'}",0.677981
2,"{'C': 0.1, 'kernel': 'linear'}",0.770927
3,"{'C': 1, 'kernel': 'linear'}",0.803804
4,"{'C': 10, 'kernel': 'linear'}",0.809935
5,"{'C': 100, 'kernel': 'linear'}",0.805936


In [14]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2, max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3

RESULTS FOR POLY DEGREE=2 KERNEL
Best parameters set found:
{'C': 0.5, 'gamma': 0.5}
Score with best parameters:
0.7323616921234061

All scores on the grid:


D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


,params,mean_test_score
0,"{'C': 0.05, 'gamma': 0.05}",0.514929
1,"{'C': 0.05, 'gamma': 0.5}",0.694418
2,"{'C': 0.05, 'gamma': 5.0}",0.449710
3,"{'C': 0.5, 'gamma': 0.05}",0.650791
4,"{'C': 0.5, 'gamma': 0.5}",0.732362
5,"{'C': 0.5, 'gamma': 5.0}",0.473427
6,"{'C': 5, 'gamma': 0.05}",0.691575
7,"{'C': 5, 'gamma': 0.5}",0.464536
8,"{'C': 5, 'gamma': 5.0}",0.473427


In [15]:
# parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='rbf', max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3

RESULTS FOR rbf KERNEL
Best parameters set found:
{'C': 5, 'gamma': 0.5}
Score with best parameters:
0.8698248637974113

All scores on the grid:


D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


,params,mean_test_score
0,"{'C': 0.5, 'gamma': 0.005}",0.496890
1,"{'C': 0.5, 'gamma': 0.05}",0.661009
2,"{'C': 0.5, 'gamma': 0.5}",0.804956
3,"{'C': 0.5, 'gamma': 5}",0.854274
4,"{'C': 5, 'gamma': 0.005}",0.590009
5,"{'C': 5, 'gamma': 0.05}",0.680203
6,"{'C': 5, 'gamma': 0.5}",0.869825
7,"{'C': 5, 'gamma': 5}",0.861827
8,"{'C': 50, 'gamma': 0.005}",0.642084
9,"{'C': 50, 'gamma': 0.05}",0.808337


In [12]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='rbf', C=5, gamma=0.5).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))

Execution time =  6.735419127941132
Highest score =  0.8898365316275764


### SVM PPG FILTERED PCA

In [13]:
ppg = pd.read_csv("../DATA_CSV/PPG_filtered_noscaled.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ppg_df, test_ppg_df = train_test_split(ppg, test_size=0.2, stratify=ppg['Label'], random_state=123)

ppg_mat_train, label_ppg_train = create_array(train_ppg_df) 
ppg_mat_test, label_ppg_test   = create_array(test_ppg_df) 

num_components = 0.95
pca2 = PCA(n_components = num_components)
pca2.fit(ecg_mat_train)
ecg_mat_train = pca2.transform(ecg_mat_train)
ecg_mat_test = pca2.transform(ecg_mat_test)

X_train = ppg_mat_train
y_train = label_ppg_train
X_test = ppg_mat_test
y_test = label_ppg_test

from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

In [17]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

RESULTS FOR LINEAR KERNEL
Best parameters set found:
{'C': 10, 'kernel': 'linear'}
Score with best parameters:
0.8099347291272042
All scores on the grid:


,params,mean_test_score
0,"{'C': 0.001, 'kernel': 'linear'}",0.495290
1,"{'C': 0.01, 'kernel': 'linear'}",0.677981
2,"{'C': 0.1, 'kernel': 'linear'}",0.770927
3,"{'C': 1, 'kernel': 'linear'}",0.803804
4,"{'C': 10, 'kernel': 'linear'}",0.809935
5,"{'C': 100, 'kernel': 'linear'}",0.805936


In [18]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2, max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3

RESULTS FOR POLY DEGREE=2 KERNEL
Best parameters set found:
{'C': 0.5, 'gamma': 0.5}
Score with best parameters:
0.7323616921234061

All scores on the grid:


D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


,params,mean_test_score
0,"{'C': 0.05, 'gamma': 0.05}",0.514929
1,"{'C': 0.05, 'gamma': 0.5}",0.694418
2,"{'C': 0.05, 'gamma': 5.0}",0.449710
3,"{'C': 0.5, 'gamma': 0.05}",0.650791
4,"{'C': 0.5, 'gamma': 0.5}",0.732362
5,"{'C': 0.5, 'gamma': 5.0}",0.473427
6,"{'C': 5, 'gamma': 0.05}",0.691575
7,"{'C': 5, 'gamma': 0.5}",0.464536
8,"{'C': 5, 'gamma': 5.0}",0.473427


In [19]:
# parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='rbf', max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3

RESULTS FOR rbf KERNEL
Best parameters set found:
{'C': 5, 'gamma': 0.5}
Score with best parameters:
0.8698248637974113

All scores on the grid:


D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


,params,mean_test_score
0,"{'C': 0.5, 'gamma': 0.005}",0.496890
1,"{'C': 0.5, 'gamma': 0.05}",0.661009
2,"{'C': 0.5, 'gamma': 0.5}",0.804956
3,"{'C': 0.5, 'gamma': 5}",0.854274
4,"{'C': 5, 'gamma': 0.005}",0.590009
5,"{'C': 5, 'gamma': 0.05}",0.680203
6,"{'C': 5, 'gamma': 0.5}",0.869825
7,"{'C': 5, 'gamma': 5}",0.861827
8,"{'C': 50, 'gamma': 0.005}",0.642084
9,"{'C': 50, 'gamma': 0.05}",0.808337


In [14]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='rbf', C=5, gamma=0.5).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))

Execution time =  6.6009803700447085
Highest score =  0.8898365316275764


### SVM PPG noFILTERED

In [15]:
ppg = pd.read_csv("../DATA_CSV/PPG_nofiltered_noscaled.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ppg_df, test_ppg_df = train_test_split(ppg, test_size=0.2, stratify=ppg['Label'], random_state=123)

ppg_mat_train, label_ppg_train = create_array(train_ppg_df) 
ppg_mat_test, label_ppg_test   = create_array(test_ppg_df) 

X_train = ppg_mat_train
y_train = label_ppg_train
X_test = ppg_mat_test
y_test = label_ppg_test

from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

In [21]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

RESULTS FOR LINEAR KERNEL
Best parameters set found:
{'C': 10, 'kernel': 'linear'}
Score with best parameters:
0.8383523710626515
All scores on the grid:


,params,mean_test_score
0,"{'C': 0.001, 'kernel': 'linear'}",0.641485
1,"{'C': 0.01, 'kernel': 'linear'}",0.789547
2,"{'C': 0.1, 'kernel': 'linear'}",0.827535
3,"{'C': 1, 'kernel': 'linear'}",0.836362
4,"{'C': 10, 'kernel': 'linear'}",0.838352
5,"{'C': 100, 'kernel': 'linear'}",0.830997


In [22]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2,max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3

RESULTS FOR POLY DEGREE=2 KERNEL
Best parameters set found:
{'C': 5, 'gamma': 0.05}
Score with best parameters:
0.858515057113188

All scores on the grid:


D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


,params,mean_test_score
0,"{'C': 0.05, 'gamma': 0.05}",0.528816
1,"{'C': 0.05, 'gamma': 0.5}",0.858429
2,"{'C': 0.05, 'gamma': 5.0}",0.636899
3,"{'C': 0.5, 'gamma': 0.05}",0.828055
4,"{'C': 0.5, 'gamma': 0.5}",0.824247
5,"{'C': 0.5, 'gamma': 5.0}",0.452146
6,"{'C': 5, 'gamma': 0.05}",0.858515
7,"{'C': 5, 'gamma': 0.5}",0.579007
8,"{'C': 5, 'gamma': 5.0}",0.444271


In [23]:
# parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='rbf', max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3

KeyboardInterrupt: 

In [16]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='poly',degree = 2, C=5, gamma=0.05).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))

Execution time =  7.172445592880249
Highest score =  0.8670820353063343


### SVM PPG FILTERED PCA

In [17]:
ppg = pd.read_csv("../DATA_CSV/PPG_nofiltered_noscaled.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ppg_df, test_ppg_df = train_test_split(ppg, test_size=0.2, stratify=ppg['Label'], random_state=123)

ppg_mat_train, label_ppg_train = create_array(train_ppg_df) 
ppg_mat_test, label_ppg_test   = create_array(test_ppg_df) 

num_components = 0.95
pca2 = PCA(n_components = num_components)
pca2.fit(ecg_mat_train)
ecg_mat_train = pca2.transform(ecg_mat_train)
ecg_mat_test = pca2.transform(ecg_mat_test)

X_train = ppg_mat_train
y_train = label_ppg_train
X_test = ppg_mat_test
y_test = label_ppg_test

from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

In [25]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

RESULTS FOR LINEAR KERNEL
Best parameters set found:
{'C': 10, 'kernel': 'linear'}
Score with best parameters:
0.8383523710626515
All scores on the grid:


,params,mean_test_score
0,"{'C': 0.001, 'kernel': 'linear'}",0.641485
1,"{'C': 0.01, 'kernel': 'linear'}",0.789547
2,"{'C': 0.1, 'kernel': 'linear'}",0.827535
3,"{'C': 1, 'kernel': 'linear'}",0.836362
4,"{'C': 10, 'kernel': 'linear'}",0.838352
5,"{'C': 100, 'kernel': 'linear'}",0.830997


In [26]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2, max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3

RESULTS FOR POLY DEGREE=2 KERNEL
Best parameters set found:
{'C': 5, 'gamma': 0.05}
Score with best parameters:
0.858515057113188

All scores on the grid:


D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


,params,mean_test_score
0,"{'C': 0.05, 'gamma': 0.05}",0.528816
1,"{'C': 0.05, 'gamma': 0.5}",0.858429
2,"{'C': 0.05, 'gamma': 5.0}",0.636899
3,"{'C': 0.5, 'gamma': 0.05}",0.828055
4,"{'C': 0.5, 'gamma': 0.5}",0.824247
5,"{'C': 0.5, 'gamma': 5.0}",0.452146
6,"{'C': 5, 'gamma': 0.05}",0.858515
7,"{'C': 5, 'gamma': 0.5}",0.579007
8,"{'C': 5, 'gamma': 5.0}",0.444271


In [27]:
# parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='rbf', max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
D:\anaconda3

RESULTS FOR rbf KERNEL
Best parameters set found:
{'C': 5, 'gamma': 0.5}
Score with best parameters:
0.8945136725510556

All scores on the grid:


D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


,params,mean_test_score
0,"{'C': 0.5, 'gamma': 0.005}",0.500087
1,"{'C': 0.5, 'gamma': 0.05}",0.845621
2,"{'C': 0.5, 'gamma': 0.5}",0.879716
3,"{'C': 0.5, 'gamma': 5}",0.870543
4,"{'C': 5, 'gamma': 0.005}",0.764019
5,"{'C': 5, 'gamma': 0.05}",0.871841
6,"{'C': 5, 'gamma': 0.5}",0.894514
7,"{'C': 5, 'gamma': 5}",0.876428
8,"{'C': 50, 'gamma': 0.005}",0.839477
9,"{'C': 50, 'gamma': 0.05}",0.834372


In [18]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='rbf', C=5, gamma=0.5).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))

Execution time =  5.954750497341156
Highest score =  0.9100034614053306
